In [1]:
# -*- coding: utf-8 -*-
"""
@Author: Guan-Fu Liu 
Created on Sep. 24, 2023
Last modified on Sep. 24, 2023
Thanks to Mingyu Li for his help in fix some bugs in this code.

List strong lines in wavelength region at a specific temperature. 
A line table file (xlsx file) will generated.

Remove the element_dict, since the pyatomdab provides pyatomdb.atomic.Ztoelsymb(Z), which
will convert Z to corresponding element symbol (such as 6 -> C). 
Remove the int_to_roman function, since the pyatomdb provides atomic.spectroscopic_name(Z,z1), 
which convert (Z, Z1) to corresponding emission line symbole (such as (6, 5) -> C V).
"""
import pyatomdb
import pandas as pd
import numpy as np

# Some essential inputs
## $T_{\mathrm{e}}$, the equilibrium temperature.
You can get a rough estimate of it by fitting via SPEX.
## Input excel files
Here, it is "Line-Table-0822960101-90-95-3.0.xlsx"

The input line table file should be like the following:
```excel
No.	Estimated Wavelength (Angstrom)	Estimated Width(Angstrom)
1	9.11	0.05
2	9.26	0.05
3	9.41	0.05
4	10.34	0.05
5	12.1	0.15
6	12.41	0.05
7	12.88	0.1
8	13.38	0.05
9	14.15	0.05
10	14.29	0.07
11	15.02	0.08
12	16.81	0.07
13	17.05	0.1
14	18.97	0.07
```
The first column is the number, estimated wavelength is the second and estimated width is the third. 
Note that the column names need not to be strictly follow the example given above. This code will not check the column names like "No.", "Estimated Wavelength (Angstrom)" and "Estimated Width(Angstrom)". However, the wavelength and width should be of the unit angstrom.
# Input response file
For RGS of XMM-Newton:
```python
sess.set_response('P0822960101R1S004RSPMAT1003.FIT')
sess.set_response('A0822960101R3O1I90E95S3.0.rsp')
```
For MEG of Chandra
```python
sess.set_response(rmf = 'aciss_meg1_cy22.grmf', arf = 'aciss_meg1_cy22.garf')
```

If you do not input a **response file**, you can still get strong lines. However, it will make some differences between the strong lines returned with and without providing a response file. 

# line_num: how many strongest lines you would like to get.
line_num = 1 : only the strongest line will be returned. 
(line_num = 1 is OK for most cases and you do not need too many lines)

In [21]:
hc = 12.3984428  # Convert energy in keV to in angstrom
# Energy in Angstrom = hc/(Energy in keV) 
table_file = 'Line-Table-Combined.xlsx'  # The name of the input file
df = pd.read_excel(table_file, usecols=range(3))

# declare the Collisional Ionization Equilibrium session
sess = pyatomdb.spectrum.CIESession()

# set response
# sess.set_response(rmf = 'aciss_meg1_cy22.grmf', arf = 'aciss_meg1_cy22.garf')
sess.set_response('MRK1216-combined.rsp')  # Comment this line if you do not have a response file.

Te = 0.7  # Unit: keV, Temperature
n = len(df.iloc[:, 0]) # How many lines.
line_num = 1  # How many strongest lines you want. 
# The default value is 1, which means only return the strongest lines

if sess.rmffile == False: 
    print("Response is not set.")
    for j in range(line_num):
        strong_lines = pd.DataFrame(
            {'The No. %d Strongest line \n(Angstrom)'% (j+1) : pd.Series(index=np.arange(n), dtype='float'), 
                             'The No. %d Strongest line \n(keV)' % (j+1) : pd.Series(index=np.arange(n), dtype='float'), 
                             'Ion': pd.Series(index=np.arange(n), dtype='str'), 
                             'Iso-el. seq.': pd.Series(index=np.arange(n), dtype='str')})
        for i in range(n):
            row = df.iloc[i, :]
            llist = sess.return_linelist(Te = Te, specrange=[row.iloc[1]-row.iloc[2], row.iloc[1]+row.iloc[2]], specunit="A",
                                    teunit="keV", apply_aeff=False)
            llist.sort(order='Epsilon')
            llist = llist[::-1]
            strong_lines.iloc[i, 0] = llist[j]['Lambda']
            strong_lines.iloc[i, 1] = hc/llist[j]['Lambda']
            strong_lines.iloc[i, 2] = pyatomdb.atomic.spectroscopic_name(llist[j]['Element'], llist[j]['Ion'])
            strong_lines.iloc[i, 3] = pyatomdb.atomic.Ztoelsymb(llist[j]['Element']-llist[j]['Ion']+1)
        strong_lines.round(decimals=2)
        df = pd.concat([df, strong_lines], axis=1)
else:
    print("Response is set.")
    for j in range(line_num):
        strong_lines = pd.DataFrame(
            {'The No. %d Strongest line \n(Angstrom)'% (j+1) : pd.Series(index=np.arange(n), dtype='float'), 
                             'The No. %d Strongest line \n(keV)' % (j+1) : pd.Series(index=np.arange(n), dtype='float'), 
                             'Ion': pd.Series(index=np.arange(n), dtype='str'), 
                             'Iso-el. seq.': pd.Series(index=np.arange(n), dtype='str')})
        for i in range(n):
            row = df.iloc[i, :]
            llist = sess.return_linelist(Te = Te, specrange=[row.iloc[1]-row.iloc[2], row.iloc[1]+row.iloc[2]], specunit="A",
                                    teunit="keV", apply_aeff=True)
            llist.sort(order='Epsilon')
            llist = llist[::-1]
            strong_lines.iloc[i, 0] = llist[j]['Lambda']
            strong_lines.iloc[i, 1] = hc/llist[j]['Lambda']
            strong_lines.iloc[i, 2] = pyatomdb.atomic.spectroscopic_name(llist[j]['Element'], llist[j]['Ion'])
            strong_lines.iloc[i, 3] = pyatomdb.atomic.Ztoelsymb(llist[j]['Element']-llist[j]['Ion']+1)
        strong_lines = strong_lines.round(decimals=2)
        df = pd.concat([df, strong_lines], axis=1)

Response is set.


In [22]:
df = df.sort_values(by=[df.columns[3]])
df[df.columns[0]] = np.arange(1, n+1, 1)
final_table = "Final-%s"%table_file
df.to_excel(final_table, index=False)

In [18]:
df

,No.,Estimated Wavelength \n(Angstrom),Estimated Width\n(Angstrom),The No. 1 Strongest line \n(Angstrom),The No. 1 Strongest line \n(keV),Ion,Iso-el. seq.
0,1,9.18,0.05,9.17,1.35,Mg XI,He
1,2,9.83,0.05,9.86,1.26,Fe XIX,O
2,3,10.69,0.07,10.66,1.16,Fe XVII,Ne
3,4,12.10,0.15,12.13,1.02,Ne X,H
4,5,12.26,0.10,12.27,1.01,Fe XVII,Ne
5,6,12.41,0.10,12.44,1.00,Ni XIX,Ne
6,7,12.85,0.10,12.85,0.97,Fe XX,N
7,8,13.49,0.10,13.52,0.92,Fe XIX,O
8,9,13.79,0.10,13.80,0.90,Fe XIX,O
9,10,14.05,0.10,13.95,0.89,Fe XVIII,F


In [23]:
help(sess.return_linelist)

Help on method return_linelist in module pyatomdb.spectrum:

return_linelist(Te, specrange, specunit='A', teunit='keV', apply_aeff=False, nearest=False, apply_binwidth=False) method of pyatomdb.spectrum.CIESession instance
    Get the list of line emissivities vs wavelengths
    
    
    Parameters
    ----------
    Te : float
      Temperature in keV or K
    specrange : [float, float]
      Minimum and maximum values for interval in which to search
    specunit : {'Angstrom','keV'}
      Units for specrange
    teunit : {'keV' , 'K'}
      Units of te (kev or K, default keV)
    apply_aeff : bool
      If true, apply the effective area to the lines in the linelist to
      modify their intensities.
    nearest : bool
      Return spectrum at nearest tabulated temperature, without interpolation
    apply_binwidth : bool
      Divide the line emissivity by the width of the bin they occupy
      to give emissivity per angstrom or per keV.
    
    Returns
    -------
    linelist : ar